# SPR 2026 - TF-IDF Treated (Processamento Avançado)

## ✅ Funciona 100% OFFLINE

**Este notebook aplica pipeline completo de tratamento de texto:**
- Limpeza e normalização
- Stop words customizadas (mantendo termos médicos)
- Extração de features BI-RADS
- Class weights para desbalanceamento

---
### Modelos combinados:
1. LinearSVC (0.77885)
2. SGDClassifier (0.75019)
3. LogisticRegression (0.72935)

---
### Configuração Kaggle:
1. Settings → **Internet OFF**
2. Settings → Accelerator → **None** (não precisa GPU)
3. Run All

---

In [ ]:
# ===== SPR 2026 - TF-IDF TREATED (PROCESSAMENTO AVANÇADO) =====

import os
import re
import numpy as np
import pandas as pd
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack, csr_matrix

print("="*60)
print("SPR 2026 - TF-IDF Treated (Processamento Avançado)")
print("="*60)

# ==== CONFIGURAÇÕES ====
SEED = 42
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'

np.random.seed(SEED)

In [ ]:
# ==== CARREGAR DADOS ====
print("\n[1/8] Carregando dados...")
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')

print(f"    Train: {train.shape}")
print(f"    Test: {test.shape}")

# Distribuição original
print("\nDistribuição das classes:")
class_dist = train['label'].value_counts().sort_index()
for label, count in class_dist.items():
    print(f"    Classe {label}: {count:5d} ({100*count/len(train):.1f}%)")

In [ ]:
# ==== STOP WORDS CUSTOMIZADAS ====
print("\n[2/8] Definindo stop words customizadas...")

# Stop words genéricas em português (MANTER termos médicos!)
STOP_WORDS = {
    # Artigos
    'o', 'a', 'os', 'as', 'um', 'uma', 'uns', 'umas',
    # Preposições
    'de', 'da', 'do', 'das', 'dos', 'em', 'na', 'no', 'nas', 'nos',
    'para', 'por', 'pela', 'pelo', 'pelas', 'pelos',
    'com', 'sem', 'sob', 'sobre', 'entre', 'ate', 'até',
    # Conjunções
    'e', 'ou', 'mas', 'porem', 'porém', 'contudo', 'entretanto',
    'que', 'quando', 'como', 'se', 'porque', 'pois',
    # Pronomes
    'eu', 'tu', 'ele', 'ela', 'nos', 'nós', 'vos', 'eles', 'elas',
    'me', 'te', 'se', 'lhe', 'lhes',
    'meu', 'minha', 'meus', 'minhas', 'seu', 'sua', 'seus', 'suas',
    'esse', 'essa', 'esses', 'essas', 'este', 'esta', 'estes', 'estas',
    'isso', 'isto', 'aquele', 'aquela', 'aqueles', 'aquelas', 'aquilo',
    # Verbos auxiliares
    'ser', 'estar', 'ter', 'haver', 'foi', 'era', 'é', 'são',
    'está', 'estão', 'tem', 'têm', 'há',
    # Advérbios comuns
    'não', 'nao', 'sim', 'já', 'ja', 'ainda', 'também', 'tambem',
    'muito', 'pouco', 'mais', 'menos', 'bem', 'mal',
    'aqui', 'ali', 'lá', 'la', 'onde', 'quando',
    # Outros
    'ao', 'aos', 'pela', 'pelo', 'pelas', 'pelos',
    'qual', 'quais', 'quanto', 'quanta', 'quantos', 'quantas',
    'todo', 'toda', 'todos', 'todas', 'outro', 'outra', 'outros', 'outras',
    'mesmo', 'mesma', 'mesmos', 'mesmas',
    'sendo', 'sido', 'tendo', 'tido',
    # Pontuação e números isolados
    'ii', 'iii', 'iv', 'vi', 'vii', 'viii', 'ix', 'xi', 'xii',
}

# Termos médicos para NUNCA remover
MEDICAL_TERMS = {
    # BI-RADS
    'birads', 'bi-rads', 'bi', 'rads', 'categoria',
    # Achados
    'nodulo', 'nódulo', 'massa', 'lesão', 'lesao', 'tumor',
    'calcificação', 'calcificacao', 'calcificações', 'microcalcificação',
    'assimetria', 'assimétrica', 'distorção', 'distorcao',
    'densidade', 'espessamento', 'retração', 'retracao',
    # Características
    'benigno', 'benigna', 'maligno', 'maligna', 'suspeito', 'suspeita',
    'espiculado', 'espiculada', 'irregular', 'circunscrito', 'circunscrita',
    'lobulado', 'lobulada', 'oval', 'redondo', 'redonda',
    'indistinto', 'indistinta', 'obscurecido', 'microlobulado',
    # Localização
    'mama', 'mamas', 'mamária', 'mamario', 'bilateral', 'unilateral',
    'direita', 'esquerda', 'superior', 'inferior', 'lateral', 'medial',
    'retroareolar', 'axilar', 'axila', 'quadrante',
    # Tecido
    'parênquima', 'parenquima', 'fibroglandular', 'adiposo', 'gorduroso',
    'denso', 'densa', 'heterogêneo', 'heterogeneo', 'homogêneo',
    # Linfonodos
    'linfonodo', 'linfonodos', 'gânglio', 'ganglio', 'adenopatia',
    # Procedimentos
    'mamografia', 'ultrassom', 'ultrassonografia', 'biópsia', 'biopsia',
    'aspiração', 'aspiracao', 'punção', 'puncao', 'core',
    # Conclusões
    'negativo', 'negativa', 'normal', 'inconclusivo',
    'provavelmente', 'altamente', 'confirmado', 'confirmada',
    # Medidas
    'cm', 'mm', 'centímetros', 'milímetros',
}

# Remover termos médicos das stop words
STOP_WORDS = STOP_WORDS - MEDICAL_TERMS
print(f"    Stop words: {len(STOP_WORDS)} palavras")
print(f"    Termos médicos protegidos: {len(MEDICAL_TERMS)} palavras")

In [ ]:
# ==== FUNÇÕES DE PROCESSAMENTO ====
print("\n[3/8] Definindo pipeline de processamento...")

# Dicionário de abreviações médicas
ABBREVIATIONS = {
    r'\bqse\b': 'quadrante superior externo',
    r'\bqsi\b': 'quadrante superior interno',
    r'\bqie\b': 'quadrante inferior externo',
    r'\bqii\b': 'quadrante inferior interno',
    r'\busg\b': 'ultrassonografia',
    r'\bus\b': 'ultrassom',
    r'\bmmg\b': 'mamografia',
    r'\bmamo\b': 'mamografia',
    r'\bln\b': 'linfonodo',
    r'\blns\b': 'linfonodos',
    r'\bca\b': 'câncer',
    r'\bcm\b': 'centímetros',
    r'\bmm\b': 'milímetros',
    r'\bd\b': 'direita',
    r'\be\b': 'esquerda',
    r'\bdir\b': 'direita',
    r'\besq\b': 'esquerda',
    r'\bsup\b': 'superior',
    r'\binf\b': 'inferior',
    r'\blat\b': 'lateral',
    r'\bmed\b': 'medial',
    r'\bc/\b': 'com',
    r'\bs/\b': 'sem',
}

# Padrões BI-RADS para extração
BIRADS_PATTERNS = [
    (r'bi-?rads?\s*:?\s*([0-6])', 'birads_explicit'),
    (r'categoria\s*:?\s*([0-6])', 'birads_categoria'),
    (r'classe\s*:?\s*([0-6])', 'birads_classe'),
]

def expand_abbreviations(text):
    """Expande abreviações médicas comuns."""
    for pattern, replacement in ABBREVIATIONS.items():
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
    return text

def normalize_numbers(text):
    """Normaliza números e medidas."""
    # Converte medidas para padrão
    text = re.sub(r'(\d+)[,\.](\d+)\s*(cm|mm)', r'\1\2\3', text)
    # Remove IDs e números longos (> 4 dígitos)
    text = re.sub(r'\b\d{5,}\b', '', text)
    # Mantém medidas mas remove outros números isolados
    text = re.sub(r'(?<!\d)\b\d{1,2}\b(?!\s*(cm|mm|anos?))', '', text)
    return text

def remove_boilerplate(text):
    """Remove seções padrão de laudos."""
    # Remove datas
    text = re.sub(r'\d{1,2}[/\-]\d{1,2}[/\-]\d{2,4}', '', text)
    # Remove horários
    text = re.sub(r'\d{1,2}:\d{2}(:\d{2})?', '', text)
    # Remove cabeçalhos comuns
    patterns = [
        r'laudo\s*:',
        r'paciente\s*:',
        r'data\s*:',
        r'médico\s*:',
        r'crm\s*:?\s*\d+',
        r'\bdr\.?\s+\w+',
        r'assinatura\s*:?',
        r'\bobs\s*:?',
    ]
    for pattern in patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    return text

def extract_birads_features(text):
    """Extrai menções explícitas de BI-RADS."""
    features = {
        'birads_mentioned': 0,
        'birads_value': -1,  # -1 = não encontrado
    }
    
    text_lower = text.lower()
    
    for pattern, name in BIRADS_PATTERNS:
        match = re.search(pattern, text_lower)
        if match:
            features['birads_mentioned'] = 1
            features['birads_value'] = int(match.group(1))
            break
    
    return features

def extract_medical_features(text):
    """Extrai features estruturadas do texto."""
    text_lower = text.lower()
    
    features = {
        # Achados principais
        'has_nodulo': 1 if re.search(r'n[óo]dulo|massa|les[ãa]o', text_lower) else 0,
        'has_calcificacao': 1 if re.search(r'calcifica[çc][ãa]o|microcalc', text_lower) else 0,
        'has_assimetria': 1 if re.search(r'assim[ée]tr', text_lower) else 0,
        'has_distorcao': 1 if re.search(r'distor[çc][ãa]o', text_lower) else 0,
        
        # Características
        'is_benigno': 1 if re.search(r'benign[oa]|provavelmente\s+benign', text_lower) else 0,
        'is_maligno': 1 if re.search(r'malign[oa]|suspeito|alta\s+suspeita', text_lower) else 0,
        'is_normal': 1 if re.search(r'normal|negativ[oa]|sem\s+(achados|altera)', text_lower) else 0,
        'is_inconclusivo': 1 if re.search(r'inconclusiv|adicional|complement', text_lower) else 0,
        
        # Morfologia
        'has_espiculado': 1 if re.search(r'espiculad|irregular|indistint', text_lower) else 0,
        'has_circunscrito': 1 if re.search(r'circunscrit|bem\s+delimitad|regular', text_lower) else 0,
        
        # Localização
        'is_bilateral': 1 if re.search(r'bilateral|ambas', text_lower) else 0,
        'has_axila': 1 if re.search(r'axil|linfono', text_lower) else 0,
        
        # Densidade
        'has_denso': 1 if re.search(r'dens[oa]|heterog[êe]ne', text_lower) else 0,
        'has_adiposo': 1 if re.search(r'adipos|gordur|lipom', text_lower) else 0,
        
        # Tamanho do texto
        'text_length': len(text),
        'word_count': len(text.split()),
    }
    
    # Adicionar features de BI-RADS
    birads = extract_birads_features(text)
    features.update(birads)
    
    return features

def remove_stopwords(text):
    """Remove stop words mantendo termos médicos."""
    words = text.split()
    return ' '.join([w for w in words if w.lower() not in STOP_WORDS])

def full_preprocess(text):
    """Pipeline completo de processamento."""
    if pd.isna(text):
        return ""
    
    text = str(text).lower()
    
    # 1. Remove boilerplate (cabeçalhos, datas)
    text = remove_boilerplate(text)
    
    # 2. Expande abreviações
    text = expand_abbreviations(text)
    
    # 3. Normaliza números
    text = normalize_numbers(text)
    
    # 4. Remove caracteres especiais (manter hífen e ponto)
    text = re.sub(r'[^a-záàâãéêíóôõúüç\s\-\.]', ' ', text)
    
    # 5. Normaliza espaços
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 6. Remove stop words
    text = remove_stopwords(text)
    
    return text

# Testar
sample = train.iloc[0]['text']
print(f"Original ({len(sample)} chars): {sample[:80]}...")
processed = full_preprocess(sample)
print(f"Processado ({len(processed)} chars): {processed[:80]}...")

In [ ]:
# ==== APLICAR PROCESSAMENTO ====
print("\n[4/8] Aplicando processamento...")

# Processar textos
X_train_text = train['text'].apply(full_preprocess)
X_test_text = test['text'].apply(full_preprocess)
y_train = train['label'].values

# Extrair features estruturadas
print("    Extraindo features médicas...")
train_features = train['text'].apply(extract_medical_features).apply(pd.Series)
test_features = test['text'].apply(extract_medical_features).apply(pd.Series)

print(f"\n    Textos processados: {len(X_train_text)} train, {len(X_test_text)} test")
print(f"    Features extraídas: {train_features.shape[1]} colunas")
print(f"    Features: {list(train_features.columns)}")

In [ ]:
# ==== TF-IDF VETORIZAÇÃO ====
print("\n[5/8] Vetorização TF-IDF...")

# TF-IDF com vocabulário otimizado
tfidf = TfidfVectorizer(
    max_features=15000,
    ngram_range=(1, 3),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\b[a-záàâãéêíóôõúüç]{2,}\b',  # Mínimo 2 caracteres
)

X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)

print(f"    Vocabulary size: {len(tfidf.vocabulary_)}")
print(f"    TF-IDF shape: {X_train_tfidf.shape}")

# Combinar TF-IDF + features estruturadas
print("\n    Combinando TF-IDF + features estruturadas...")
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
test_features_scaled = scaler.transform(test_features)

X_train_combined = hstack([X_train_tfidf, csr_matrix(train_features_scaled)])
X_test_combined = hstack([X_test_tfidf, csr_matrix(test_features_scaled)])

print(f"    Shape final: {X_train_combined.shape}")

In [ ]:
# ==== CLASS WEIGHTS CUSTOMIZADOS ====
print("\n[6/8] Calculando class weights...")

# Class weights baseados na frequência inversa
class_counts = train['label'].value_counts().sort_index()
total = len(train)
n_classes = len(class_counts)

# Fórmula: balanced weights com boost para classes críticas (5, 6)
weights = {}
for label, count in class_counts.items():
    base_weight = total / (n_classes * count)
    # Boost extra para classes críticas (5, 6)
    if label >= 5:
        base_weight *= 1.5  # 50% extra
    weights[label] = base_weight

print("    Class weights calculados:")
for label, weight in sorted(weights.items()):
    print(f"        Classe {label}: {weight:.3f}")

In [ ]:
# ==== TREINAR MODELOS TOP 3 ====
print("\n[7/8] Treinando Top 3 modelos...")

# 1. LinearSVC (melhor single model: 0.77885)
print("\n--- LinearSVC ---")
svc = LinearSVC(
    C=0.5,
    loss='squared_hinge',
    max_iter=3000,
    class_weight=weights,
    dual='auto',
    random_state=SEED
)
svc_calibrated = CalibratedClassifierCV(svc, cv=3)
svc_calibrated.fit(X_train_combined, y_train)
print("    Treinado!")

# 2. SGDClassifier (segundo melhor: 0.75019)
print("\n--- SGDClassifier ---")
sgd = SGDClassifier(
    loss='modified_huber',
    alpha=1e-4,
    penalty='l2',
    max_iter=1000,
    class_weight=weights,
    random_state=SEED,
    n_jobs=-1
)
sgd.fit(X_train_combined, y_train)
print("    Treinado!")

# 3. LogisticRegression (terceiro: 0.72935)
print("\n--- LogisticRegression ---")
lr = LogisticRegression(
    C=1.0,
    solver='lbfgs',
    max_iter=1000,
    class_weight=weights,
    multi_class='multinomial',
    random_state=SEED,
    n_jobs=-1
)
lr.fit(X_train_combined, y_train)
print("    Treinado!")

In [ ]:
# ==== CROSS-VALIDATION ====
print("\n--- Cross-Validation (5-fold) ---")

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

# Re-treinar para CV
svc_cv = CalibratedClassifierCV(
    LinearSVC(C=0.5, class_weight=weights, dual='auto', random_state=SEED), cv=3
)
sgd_cv = SGDClassifier(
    loss='modified_huber', alpha=1e-4, class_weight=weights, random_state=SEED
)
lr_cv = LogisticRegression(
    C=1.0, class_weight=weights, multi_class='multinomial', random_state=SEED
)

svc_scores = cross_val_score(svc_cv, X_train_combined, y_train, cv=cv, scoring='f1_macro')
sgd_scores = cross_val_score(sgd_cv, X_train_combined, y_train, cv=cv, scoring='f1_macro')
lr_scores = cross_val_score(lr_cv, X_train_combined, y_train, cv=cv, scoring='f1_macro')

print(f"\n    LinearSVC:     {svc_scores.mean():.5f} ± {svc_scores.std():.5f}")
print(f"    SGDClassifier: {sgd_scores.mean():.5f} ± {sgd_scores.std():.5f}")
print(f"    LogisticReg:   {lr_scores.mean():.5f} ± {lr_scores.std():.5f}")

In [ ]:
# ==== ENSEMBLE PREDICTIONS ====
print("\n[8/8] Gerando predições ensemble...")

# Predições individuais
pred_svc = svc_calibrated.predict(X_test_combined)
pred_sgd = sgd.predict(X_test_combined)
pred_lr = lr.predict(X_test_combined)

# Probabilidades
proba_svc = svc_calibrated.predict_proba(X_test_combined)
proba_sgd = sgd.predict_proba(X_test_combined)
proba_lr = lr.predict_proba(X_test_combined)

# Weighted Voting (pesos baseados no score público)
weights_voting = np.array([0.77885, 0.75019, 0.72935])
weights_voting = weights_voting / weights_voting.sum()

proba_weighted = (
    proba_svc * weights_voting[0] + 
    proba_sgd * weights_voting[1] + 
    proba_lr * weights_voting[2]
)
pred_weighted = np.argmax(proba_weighted, axis=1)

print(f"\n    LinearSVC dist:  {dict(Counter(pred_svc))}")
print(f"    Weighted dist:   {dict(Counter(pred_weighted))}")

In [ ]:
# ==== SUBMISSÕES ====
print("\n--- Gerando submissões ---")

# 1. LinearSVC tratado
submission_svc = pd.DataFrame({
    'id': test['id'],
    'label': pred_svc
})
submission_svc.to_csv('/kaggle/working/submission_linearsvc_treated.csv', index=False)
print("    submission_linearsvc_treated.csv")

# 2. Weighted Ensemble tratado
submission_weighted = pd.DataFrame({
    'id': test['id'],
    'label': pred_weighted
})
submission_weighted.to_csv('/kaggle/working/submission_weighted_treated.csv', index=False)
print("    submission_weighted_treated.csv")

# PRINCIPAL: LinearSVC (best single model)
submission_svc.to_csv('/kaggle/working/submission.csv', index=False)

print("\n" + "="*60)
print("CONCLUÍDO - submission.csv = LinearSVC Treated")
print("="*60)
print("\nProcessamento aplicado:")
print("  - Stop words customizadas (preservando termos médicos)")
print("  - Expansão de abreviações médicas")
print("  - Remoção de boilerplate (cabeçalhos, datas)")
print("  - Normalização de números")
print("  - Extração de features estruturadas (17 features)")
print("  - Class weights com boost para classes 5/6")